In [35]:
# Import Relevant Libraries
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [52]:
# Reading files in folder
direc = "Data"
all_files = os.listdir(direc)
jig_files = [file for file in all_files if "jig" in file]

y_train_back = []
y_train_front = []
x_train = []

data_types = [" fx[N]", " fy[N]", " fz[N]"]
# Extracting data from CSV and giving them labels
for file_name in jig_files:
    df = pd.read_csv(os.path.join(direc, file_name))
    data = []
    for tst in data_types:
        temp = df[tst].values
        temp = temp[0:210]
        data.append(temp)
    x_train.append(np.concatenate(data))
    
    back_res = file_name[-7:-6]
    front_res = file_name[-5:-4]
    
    if back_res == "P":
        y_train_back.append("P")
    else:
        y_train_back.append("F")
    
    if front_res == "P":
        y_train_front.append("P")
    else:
        y_train_front.append("F")

x_train = np.stack(x_train, axis=0)
y_train_front = np.stack(y_train_front, axis=0)
y_train_back = np.stack(y_train_back, axis=0)

print(x_train.shape)
print(y_train_front.shape)
print(y_train_back.shape)

(100, 630)
(100,)
(100,)


In [54]:
# NN Model Training

x_train_cpy, x_test_cpy, y_train_back_cpy, y_test_back_cpy = train_test_split(x_train, y_train_back, test_size=0.2)

parameters = {
    'hidden_layer_sizes': [(500, 250, 50, 10)],
    'activation': ['relu'],
    'solver': ['adam'],
    'alpha': [0.001],
    'learning_rate': ['constant'],
    'max_iter': [500],
}

# Create MLPClassifier instance
clf = MLPClassifier(random_state=42)

# Use GridSearchCV for parameter tuning
grid_search = GridSearchCV(clf, parameters, cv=5)
grid_search.fit(x_train_cpy, y_train_back_cpy)

# Best parameters found
print("Best parameters:", grid_search.best_params_)

# Evaluate the best model
best_model = grid_search.best_estimator_
accuracy = best_model.score(x_test_cpy, y_test_back_cpy)
print("Accuracy of the best model:", accuracy)

Best parameters: {'activation': 'relu', 'alpha': 0.001, 'hidden_layer_sizes': (500, 250, 50, 10), 'learning_rate': 'constant', 'max_iter': 500, 'solver': 'adam'}
Accuracy of the best model: 0.65


In [16]:
# Testing Zone
print(jig_files[0])
print(jig_files[0][-7:-6])
print(jig_files[0][-5:-4])

100_jig_M_P.csv
M
P
